In [10]:
!pip install pdfplumber pandas tabulate

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)


In [8]:
import pdfplumber, re, pandas as pd

In [9]:

pdf_path = "National Grid Tariff-New York.pdf"

# --- Regex patterns ---
plan_pattern = re.compile(
    r"(Service\s+Class(?:ification)?\s+No\.?\s*\d+[A-Z\-]*\s*[–-]?\s*[A-Za-z0-9\s,&]*)",
    re.IGNORECASE
)
effective_pattern = re.compile(
    r"(?:Effective\s+(?:Date|on|from)|Issued)\s*[:\-]?\s*([A-Za-z]+\s*\d{1,2},?\s*\d{4})",
    re.IGNORECASE
)
rate_pattern = re.compile(
    r"(\$?\s?\d+\.\d{2}\s*(?:per|/)\s*(?:kWh|kW|day|month|therm)|\$\s?\d+\.\d{2})",
    re.IGNORECASE
)

results = []

with pdfplumber.open(pdf_path) as pdf:
    for page_num, page in enumerate(pdf.pages, start=1):
        text = page.extract_text()
        if not text:
            continue

        lines = [l.strip() for l in text.split("\n") if l.strip()]

        # Get the last mentioned effective/issued date on this page
        eff_dates = effective_pattern.findall(text)
        current_effective = eff_dates[-1] if eff_dates else None

        current_plan = None

        for line in lines:
            # Detect new service plan name
            plan_match = plan_pattern.search(line)
            if plan_match:
                current_plan = plan_match.group(1).strip()

            # Detect tariff/rate lines
            rate_match = rate_pattern.findall(line)
            if rate_match and current_plan:
                results.append({
                    "Page": page_num,
                    "Plan Name": current_plan,
                    "Effective Date": current_effective,
                    "Tariff Line": line
                })

# Convert to DataFrame and clean duplicates
df = pd.DataFrame(results).drop_duplicates().reset_index(drop=True)

# --- Pretty printing ---
if df.empty:
    print("⚠️ No tariff data found — check if PDF is scanned or uses tables only.")
else:
    for plan in df["Plan Name"].unique():
        subset = df[df["Plan Name"] == plan]
        eff = subset["Effective Date"].iloc[0] or "Not Found"
        print(f"\n📘 {plan}")
        print(f"📅 Effective Date: {eff}")
        for _, row in subset.iterrows():
            print(f"💰 Tariff Line: {row['Tariff Line']}")


📘 Service Classification No. 1 or Service Classification No
📅 Effective Date: APRIL 27, 2009
💰 Tariff Line: establishing service or discontinuing service shall be assessed a charge of $20.00. The $20.00
💰 Tariff Line: 26.13.2 The $20.00 charge shall be applicable and limited to one charge per visit for the same
💰 Tariff Line: 26.13.3 A single $20.00 charge shall apply to multiple accounts for the same customer at the same
💰 Tariff Line: visit is exempt from the $20.00 charge.

📘 SERVICE CLASSIFICATION NO. 2-DEMAND
📅 Effective Date: OCTOBER 1, 2025
💰 Tariff Line: Distribution (per kW): $0.00 $4.25 $8.50 $12.74
💰 Tariff Line: On Peak (per kWh): $0.07420 $0.05565 $0.03710 $0.01855
💰 Tariff Line: Off Peak (per kWh): $0.03710 $0.02783 $0.01855 $0.00928
💰 Tariff Line: Super Peak (per kWh): $0.11131 $0.08348 $0.05565 $0.02783

📘 SERVICE CLASSIFICATION NO. 3
📅 Effective Date: OCTOBER 1, 2025
💰 Tariff Line: Distribution (per kW): $0.00 $3.57 $7.14 $10.71
💰 Tariff Line: On Peak (per kWh): $0.04